In [19]:
import os
import json
import pandas as pd
from IPython.display import display, HTML

In [2]:
# Input Parameters
prompt_techniques = ["base", "few_shot", "chain_of_thought", "prompt_maieutica"]


In [3]:
# Initialize DataFrame
columns = ["prompt_technique", "level", "avg_relevance", "avg_clarity", "avg_feasibility", "sub_feature_percentage"]
results_df = pd.DataFrame(columns=columns)

In [ ]:
# Process each prompt technique
for technique in prompt_techniques:
    for level in ["first_level", "second_level"]:
        eval_path = f"../data/gemini/{technique}/{level}_eval.json"
        if os.path.exists(eval_path):
            with open(eval_path, "r", encoding="utf-8") as f:
                eval_data = json.load(f)

            relevance_scores = []
            clarity_scores = []
            feasibility_scores = []
            sub_feature_count = 0

            for feature in eval_data:
                eval_result = feature["feature_eval"]
                relevance_scores.append(eval_result["relevance"])
                clarity_scores.append(eval_result["clarity"])
                feasibility_scores.append(eval_result["feasibility"])
                if eval_result["relationship"] == "Sub-feature":
                    sub_feature_count += 1

            total_features = len(eval_data)
            avg_relevance = sum(relevance_scores) / total_features
            avg_clarity = sum(clarity_scores) / total_features
            avg_feasibility = sum(feasibility_scores) / total_features
            sub_feature_percentage = (sub_feature_count / total_features) * 100

            results_df = pd.concat([results_df, pd.DataFrame.from_records([{
                "prompt_technique": technique,
                "level": level,
                "avg_relevance": avg_relevance,
                "avg_clarity": avg_clarity,
                "avg_feasibility": avg_feasibility,
                "sub_feature_percentage": sub_feature_percentage
            }])], ignore_index=True)

            # Create a separate table for each technique + level
            table_df = pd.DataFrame.from_records([{
                "avg_relevance": avg_relevance,
                "avg_clarity": avg_clarity,
                "avg_feasibility": avg_feasibility,
                "sub_feature_percentage": sub_feature_percentage
            }])
            table_path = f"../data/gemini/result_tables/{technique}_{level}_summary.csv"
            table_df.to_csv(table_path, index=False)

In [50]:
tecnica = "prompt_maieutica"
nivel = "second_level"

In [51]:
df = pd.read_csv(f"../data/gemini/result_tables/{tecnica}_{nivel}_summary.csv")
styled_html = (
    df.style
    .format("{:.2f}")
    .hide(axis="index")  # Esconde o índice
    .set_table_styles([
        {'selector': 'thead th',
         'props': [('background-color', '#007bff'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'tbody td',
         'props': [('text-align', 'center'), ('padding', '8px'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)',
         'props': [('background-color', '#f8f9fa')]},
        {'selector': 'tbody tr:nth-child(even)',
         'props': [('background-color', 'white')]},
        {'selector': 'table',
         'props': [('border-collapse', 'collapse'), ('width', '100%'), ('margin', '10px 0')]},
    ])
    .to_html()
)

# Exibindo o HTML formatado
display(HTML(styled_html))

avg_relevance,avg_clarity,avg_feasibility,sub_feature_percentage
5.00,4.99,4.92,98.00
